In [6]:
import napari
viewer = napari.Viewer()
from segment_everything.stacked_labels import StackedLabels
from pathlib import Path
from skimage.io import imread

#tnia_images_path = Path(r"D:\images")
tnia_images_path = Path(r"/home/bnorthan/images")

parent_path=Path(tnia_images_path / r'tnia-python-images/imagesc/2024_05_01_cellpose_ladybug')
im_name = '522_img_crop.png'
txt_name = '522_img_crop.txt'

'''
parent_path=Path(tnia_images_path / r'tnia-python-images/imagesc/2024_05_03_mouse_and_grid')
im_name = '6AMAH2b.png'
txt_name = '6AMAH2b.txt'
'''

img = imread(parent_path / im_name)
yolos = StackedLabels.read_yolo_txt(parent_path / txt_name)
stacked_labels = StackedLabels.from_yolo_dictionary(yolos, img, 'xywhn')


In [7]:
print(stacked_labels.mask_list[0].keys())

dict_keys(['segmentation', 'indexes', 'point_coords', 'prompt_bbox', 'area', 'predicted_iou', 'stability_score', 'image', 'keep'])


In [8]:
print(type(yolos))
print(type(stacked_labels))

<class 'list'>
<class 'segment_everything.stacked_labels.StackedLabels'>


In [9]:

from segment_everything.napari_helper import stacked_labels_to_napari
#viewer = napari.Viewer()
stacked_labels_to_napari(stacked_labels)

area 20 2500
label_num 1 71
solidity 1.0 1.0
circularity 0.7951678539102343 1
mean_intensity 39.99 182.53125
10th_percentile_intensity 0.0 136.2
mean_hue 63.75 85.0
mean_saturation 191.25 255.0
predicted_iou 1 1
stability_score 1 1


In [10]:
from segment_everything.mask_detectors.mobilesam import mobilesam_detector
# Re-segment using the MobileSAM adapter instead of the old helper function
detector = mobilesam_detector(model_type="MobileSamV2", device="cpu")
detector.set_image(img)
bboxes = stacked_labels.get_bbox_np()
mask_list = detector.segment_boxes(bboxes)
new_stacked_labels = StackedLabels(mask_list, img)

VIT checkpoint loaded successfully


In [11]:
stacked_labels_to_napari(new_stacked_labels)

area 8.0 1482.0
label_num 1 71
solidity 0.47619047619047616 1.0
circularity 0.6581912199193756 1
mean_intensity 19.3 209.22413793103448
10th_percentile_intensity 0.0 126.39999999999999
mean_hue 55.25 85.0
mean_saturation 165.75 255.0
predicted_iou 0.5542604327201843 0.9848764538764954
stability_score 0.6153846383094788 1.0
